In [3]:
import pandas as pd

In [4]:
# pip install pandas_datareader

In [5]:
import pandas_datareader as pdr
import numpy as np

In [6]:
df = pd.read_csv(r"C:\Users\91763\Downloads\Apple Stock Prediction-main\AAPL.csv")

In [9]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-09-08,113.949997,118.989998,112.680000,112.820000,111.629189,231366600
1,2020-09-09,117.260002,119.139999,115.260002,117.320000,116.081688,176940500
2,2020-09-10,120.360001,120.500000,112.500000,113.489998,112.292114,182274400
3,2020-09-11,114.570000,115.230003,110.000000,112.000000,110.817848,180860300
4,2020-09-14,114.720001,115.930000,112.800003,115.360001,114.142387,140150100


In [10]:
df2=df.reset_index()["Close"]

In [ ]:
df2

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df2=scaler.fit_transform(np.array(df2).reshape(-1,1))

In [ ]:
df.shape

In [ ]:
df2

In [ ]:
#lets split the data into train and test
training_size = int(len(df2)*0.65)
test_size = len(df2)-training_size
train_data,test_data= df2[0:training_size,:],df2[training_size:len(df2),:1]

In [ ]:
training_size,test_size

In [ ]:
# converting an array of values into a dataset matrix
import numpy
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
time_step = 1
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)
print(X_train.shape), print(y_train.shape)

In [ ]:
print(X_test.shape), print(ytest.shape)

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

In [ ]:
import tensorflow as tf

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
math.sqrt(mean_squared_error(ytest,test_predict))

In [ ]:
#shifting train predictions for plotting
look_back=1
trainPredictPlot = numpy.empty_like(df2)
trainPredictPlot[:,:]= np.nan
trainPredictPlot[look_back:len(train_predict)+look_back,:] = train_predict
#shifting test predictions for plotting
testPredictPlot = numpy.empty_like(df2)
testPredictPlot[:, :]= np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df2)-1, :] = test_predict
plt.plot(scaler.inverse_transform(df2))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()


In [ ]:
len(test_data)

In [ ]:
x_input = test_data[0: ].reshape(1,-1)

In [ ]:
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
from numpy import array

lst_output=[]
n_steps=88
i=0
while(i<10):
    
    if(len(temp_input)>1):
    
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
new_day = np.arange(1,253)
pred_day = np.arange(60,89)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(df2)

In [ ]:
df3 = df2.tolist()

In [ ]:
df3.extend(lst_output)

In [ ]:
len(df3)

In [ ]:
plt.plot(new_day,scaler.inverse_transform(df2[0:]))
plt.plot(pred_day,scaler.inverse_transform(lst_output))

In [ ]:
df3=df2.tolist()
df3.extend(lst_output)
plt.plot(df3[0:300])